In [167]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
from jproperties import Properties


from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline
import multiprocessing

In [168]:
configs = Properties()
with open('config.properties', 'rb') as config_file:
    configs.load(config_file)
csvfilepath = configs["filePath"].data

        # Read the csv File
df = pd.read_csv(csvfilepath, na_values='')
#objectProductReco = ProdReco(csvfilepath)
print(df.shape)

#correlation matrix
#corrmat = objectProductReco.corr()
#f, ax = plt.subplots(figsize=(12, 9))
#sns.heatmap(corrmat, vmax=.8, square=True);



(384, 25)


In [169]:
df.dtypes

PERSON_ID                 int64
PERSON_TYPE              object
BIRTH_DT                 object
LANGUAGE_CD              object
DIVISION                 object
CUSTOMER_SEG_FLG         object
CUSTOMER_TIER_FLG        object
PERSON_NAME              object
PARTY_TYPE               object
PRICING_ATTACHED_TO      object
PRICEITEM_CD             object
PRICELIST_ID            float64
PRICE_STATUS             object
PRICING_START_DT         object
PRICING_END_DT          float64
PRICE_ASGN_ID           float64
ACCOUNT_ID                int64
CUST_CL_CD               object
CIS_DIVISION             object
ACCOUNT_CATEGORY         object
SETUP_DT                 object
CURRENCY_CD              object
BILL_CYC_CD              object
PRODUCT_CD               object
PRODUCT_EFFECTIVE_DT     object
dtype: object

In [170]:
df.columns.tolist()

['PERSON_ID',
 'PERSON_TYPE',
 'BIRTH_DT',
 'LANGUAGE_CD',
 'DIVISION',
 'CUSTOMER_SEG_FLG',
 'CUSTOMER_TIER_FLG',
 'PERSON_NAME',
 'PARTY_TYPE',
 'PRICING_ATTACHED_TO',
 'PRICEITEM_CD',
 'PRICELIST_ID',
 'PRICE_STATUS',
 'PRICING_START_DT',
 'PRICING_END_DT',
 'PRICE_ASGN_ID',
 'ACCOUNT_ID',
 'CUST_CL_CD',
 'CIS_DIVISION',
 'ACCOUNT_CATEGORY',
 'SETUP_DT',
 'CURRENCY_CD',
 'BILL_CYC_CD',
 'PRODUCT_CD',
 'PRODUCT_EFFECTIVE_DT']

In [171]:
'PRODUCT_EFFECTIVE_DT','PRICE_ASGN_ID','PRICELIST_ID',
 'PRICING_START_DT',
 'PRICING_END_DT','BIRTH_DT',
 'CUSTOMER_SEG_FLG',
 'CUSTOMER_TIER_FLG','ACCOUNT_ID','SETUP_DT',

IndentationError: unexpected indent (<ipython-input-171-85a1ae567a42>, line 2)

In [172]:
dataset_person_item =df.groupby(by=['PERSON_ID', 'PRICEITEM_CD'],as_index=False).first()
dataset_person_item

,PERSON_ID,PRICEITEM_CD,PERSON_TYPE,BIRTH_DT,LANGUAGE_CD,DIVISION,CUSTOMER_SEG_FLG,CUSTOMER_TIER_FLG,PERSON_NAME,PARTY_TYPE,...,PRICE_ASGN_ID,ACCOUNT_ID,CUST_CL_CD,CIS_DIVISION,ACCOUNT_CATEGORY,SETUP_DT,CURRENCY_CD,BILL_CYC_CD,PRODUCT_CD,PRODUCT_EFFECTIVE_DT
0,72231957,AB_AOF,P,NaN,ENG,CAL,CORP,,"Lorrie, Johnson",PERS,...,1.100001e+09,2219073713,BKCORP,CAL,USAG,1/1/2020,USD,,NaN,NaN
1,72231957,ACCMAINT,P,NaN,ENG,CAL,CORP,,"Lorrie, Johnson",PERS,...,5.370001e+09,2219073713,BKCORP,CAL,USAG,1/1/2020,USD,,NaN,NaN
2,72231957,DISCT_KS,P,NaN,ENG,CAL,CORP,,"Lorrie, Johnson",PERS,...,5.370001e+09,2219073713,BKCORP,CAL,USAG,1/1/2020,USD,,NaN,NaN
3,72231957,DM_P1,P,NaN,ENG,CAL,CORP,,"Lorrie, Johnson",PERS,...,1.100000e+09,2219073713,BKCORP,CAL,USAG,1/1/2020,USD,,NaN,NaN
4,156312464,CHQTFR,P,NaN,ENG,NY,CORP,T1,"Universal (NY), Bearings",PERS,...,8.120000e+09,258542192,BKCORP,NY,USAG,4/9/2020,USD,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,8640624497,DOM_MIN,P,NaN,ENG,CAL,,,"Charles, M David",PERS,...,8.900000e+09,4720595602,BKCORP,CAL,USAG,1/1/2016,USD,EOM,NaN,NaN
171,8640624497,DOM_STD,P,NaN,ENG,CAL,,,"Charles, M David",PERS,...,8.900000e+09,4720595602,BKCORP,CAL,USAG,1/1/2016,USD,EOM,NaN,NaN
172,8640624497,DOM_URG,P,NaN,ENG,CAL,,,"Charles, M David",PERS,...,8.900000e+09,4720595602,BKCORP,CAL,USAG,1/1/2016,USD,EOM,NaN,NaN
173,9541308934,PP_PI,P,NaN,ENG,CAL,,,"PP, PER",ACCT,...,1.570001e+09,8202806258,BKCORP,CAL,USAG,1/1/2020,USD,,NaN,NaN


In [173]:
feature_names = ['PERSON_ID',
 'PERSON_TYPE',
 'LANGUAGE_CD',
 'DIVISION',
 'CUSTOMER_SEG_FLG',
 'CUSTOMER_TIER_FLG',
 'PARTY_TYPE',
 'PRICING_ATTACHED_TO',
 'PRICE_STATUS',
 'CUST_CL_CD',
 'CIS_DIVISION',
 'ACCOUNT_CATEGORY',
 'CURRENCY_CD',
 'BILL_CYC_CD',
 'PRODUCT_CD']
labels=['PRICEITEM_CD']
X = dataset_person_item[feature_names]
y = dataset_person_item[labels]

In [174]:

personid_list = X['PERSON_ID']


In [175]:
tobe_processed =X.iloc[:,1:]

In [176]:

MAX_OH_CARDINALITY=10
def select_oh_features(dataset):
    
    hc_features =dataset\
        .select_dtypes(['object', 'category'])\
        .apply(lambda col: col.nunique())\
        .loc[lambda x: x <= MAX_OH_CARDINALITY]\
        .index\
        .tolist()
        
    return hc_features

oh_features = select_oh_features(tobe_processed)

print(f'N oh_features: {len(oh_features)} \n')
print(', '.join(oh_features))

N oh_features: 14 

PERSON_TYPE, LANGUAGE_CD, DIVISION, CUSTOMER_SEG_FLG, CUSTOMER_TIER_FLG, PARTY_TYPE, PRICING_ATTACHED_TO, PRICE_STATUS, CUST_CL_CD, CIS_DIVISION, ACCOUNT_CATEGORY, CURRENCY_CD, BILL_CYC_CD, PRODUCT_CD


In [177]:
oh_pipeline = make_pipeline(SimpleImputer(strategy='constant'), OneHotEncoder(handle_unknown='ignore'))

In [178]:
column_transformer = ColumnTransformer(transformers=\
                                       [('oh_pipeline', oh_pipeline, select_oh_features)],
                                       n_jobs = multiprocessing.cpu_count(),
                                       remainder='drop')

In [179]:
X

,PERSON_ID,PERSON_TYPE,LANGUAGE_CD,DIVISION,CUSTOMER_SEG_FLG,CUSTOMER_TIER_FLG,PARTY_TYPE,PRICING_ATTACHED_TO,PRICE_STATUS,CUST_CL_CD,CIS_DIVISION,ACCOUNT_CATEGORY,CURRENCY_CD,BILL_CYC_CD,PRODUCT_CD
0,72231957,P,ENG,CAL,CORP,,PERS,Price List,ACTV,BKCORP,CAL,USAG,USD,,NaN
1,72231957,P,ENG,CAL,CORP,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD,,NaN
2,72231957,P,ENG,CAL,CORP,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD,,NaN
3,72231957,P,ENG,CAL,CORP,,PERS,Price List,ACTV,BKCORP,CAL,USAG,USD,,NaN
4,156312464,P,ENG,NY,CORP,T1,PERS,Price List,ACTV,BKCORP,NY,USAG,USD,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,8640624497,P,ENG,CAL,,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD,EOM,NaN
171,8640624497,P,ENG,CAL,,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD,EOM,NaN
172,8640624497,P,ENG,CAL,,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD,EOM,NaN
173,9541308934,P,ENG,CAL,,,ACCT,Account/Person,ACTV,BKCORP,CAL,USAG,USD,,NaN


In [180]:
processed_data = column_transformer.fit_transform(tobe_processed)
processed_data

array([[0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [181]:

processed_dataframe = pd.DataFrame(processed_data)

In [182]:
processed_dataframe

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
171,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
172,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
173,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [183]:
processed_dataframe.insert(0, 'PERSON_ID', personid_list, True)

In [184]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(processed_dataframe, y, random_state=0)

In [185]:
X_train

,PERSON_ID,0,1,2,3,4,5,6,7,8,...,36,37,38,39,40,41,42,43,44,45
156,8019217510,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19,1573938397,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
134,6300944774,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
74,4496994045,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
146,7518120155,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,5371984021,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
67,4483529959,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
117,5657544003,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
47,3207807940,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [187]:
personid_list_test=X_test['PERSON_ID']
X_train_New = X_train.copy()

In [188]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.loc[:,X_train.columns != 'PERSON_ID'])
X_test = scaler.transform(X_test.loc[:,X_test.columns != 'PERSON_ID'])
#y_train = scaler.fit_transform(y_train)
#X_train = scaler.fit_transform(X_train)


In [189]:
X_train_New = pd.concat([X_train_New,y_train], axis=1)
X_train_New

,PERSON_ID,0,1,2,3,4,5,6,7,8,...,37,38,39,40,41,42,43,44,45,PRICEITEM_CD
156,8019217510,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,ACHREV
19,1573938397,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,ACHPIF
134,6300944774,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,SCTB
74,4496994045,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,AB_CWN
146,7518120155,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,LTF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,5371984021,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ACHDR
67,4483529959,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,AB_AMF
117,5657544003,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,ERIBODR
47,3207807940,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PAY_DOM_URG


In [190]:
X_train_new_cross = pd.crosstab(X_train_New.PERSON_ID, X_train_New.PRICEITEM_CD).reset_index().rename_axis('',axis='columns') 
X_train_new_cross

,PERSON_ID,AB_AMF,AB_AOF,AB_CCARDS,AB_CWN,ACCMAINT,ACCOPEN,ACCOPENING,ACHCR,ACHDR,...,PROFFEE,PROFFEES,SAVING,SAVINGS,SCTB,SCTT,SDDB,ST,TFR,VAT
0,72231957,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,274010452,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,851053614,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1125945247,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1573938397,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2131825814,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2726379162,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
7,3129560053,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,3207807940,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
9,3253048158,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [191]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
#pred = knn.predict(X_test)
#print(type(pred))
#print(pred.shape)
#pred_df = pd.DataFrame(pred)
#print(pred_df.shape)
prob_pred=knn.predict_proba(X_test)
print(prob_pred.shape)
pred_prob_df = pd.DataFrame(prob_pred)
print(pred_prob_df.shape)


(44, 68)
(44, 68)


<ipython-input-191-8bbb82d9512d>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn.fit(X_train, y_train)


In [192]:
from sklearn.linear_model import LogisticRegression
logreg= LogisticRegression().fit(X_train,y_train)
pred = logreg.predict(X_test)

prob_pred=logreg.predict_proba(X_test)
print(prob_pred.shape)
pred_prob_df = pd.DataFrame(prob_pred)
print(pred_prob_df.shape)
#pred=logreg.predict_proba(X_test)
#print([i[1] for i in pred]) 

(44, 68)
(44, 68)


c:\users\reraveen\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [193]:
listclasses = logreg.classes_

In [194]:

pred_prob_df.columns =listclasses
pred_prob_df

,AB_AMF,AB_AOF,AB_CCARDS,AB_CWN,ACCMAINT,ACCOPEN,ACCOPENING,ACHCR,ACHDR,ACHPIF,...,PROFFEE,PROFFEES,SAVING,SAVINGS,SCTB,SCTT,SDDB,ST,TFR,VAT
0,0.029727,0.054557,0.030540,0.021960,0.026850,0.002136,0.007594,0.002643,0.003705,0.050045,...,0.004080,0.002595,0.005581,0.003192,0.013725,0.044956,0.008615,0.002634,0.002802,0.002636
1,0.029727,0.054557,0.030540,0.021960,0.026850,0.002136,0.007594,0.002643,0.003705,0.050045,...,0.004080,0.002595,0.005581,0.003192,0.013725,0.044956,0.008615,0.002634,0.002802,0.002636
2,0.060543,0.075745,0.015274,0.019422,0.017514,0.001203,0.005365,0.012612,0.009926,0.080154,...,0.001532,0.001214,0.004005,0.002374,0.011499,0.022258,0.014069,0.001916,0.043033,0.001188
3,0.029727,0.054557,0.030540,0.021960,0.026850,0.002136,0.007594,0.002643,0.003705,0.050045,...,0.004080,0.002595,0.005581,0.003192,0.013725,0.044956,0.008615,0.002634,0.002802,0.002636
4,0.008986,0.044365,0.009365,0.019623,0.171723,0.002377,0.121236,0.002112,0.006642,0.009711,...,0.005525,0.001805,0.005056,0.002642,0.008259,0.008265,0.019283,0.008660,0.014475,0.001615
5,0.142558,0.193790,0.005284,0.005555,0.005751,0.005292,0.003160,0.041101,0.059391,0.010406,...,0.012495,0.004334,0.007647,0.006430,0.002930,0.003541,0.003641,0.010195,0.019062,0.003949
6,0.015961,0.046000,0.006743,0.004162,0.018605,0.018741,0.015895,0.007721,0.037034,0.003942,...,0.134548,0.018411,0.008839,0.006237,0.002986,0.005128,0.001817,0.035615,0.006655,0.017108
7,0.019297,0.016236,0.006904,0.008591,0.005999,0.001543,0.002302,0.006204,0.003070,0.142036,...,0.000419,0.001548,0.001921,0.004007,0.005602,0.008831,0.006946,0.003144,0.009593,0.001482
8,0.003003,0.004237,0.004414,0.003452,0.006830,0.005427,0.005036,0.000718,0.001082,0.007877,...,0.002397,0.003364,0.001295,0.002171,0.002998,0.005696,0.002036,0.002072,0.002028,0.002911
9,0.005578,0.009742,0.002984,0.001866,0.005748,0.109301,0.005762,0.003618,0.009682,0.002668,...,0.026208,0.018411,0.003843,0.009302,0.001429,0.002084,0.000900,0.015145,0.003066,0.064479


In [208]:
type(pred_prob_df)

pandas.core.frame.DataFrame

In [213]:
personid_test_df=pd.DataFrame(personid_list_test)
personid_test_df

,PERSON_ID
54,3253048158
153,7815688111
63,4186826007
55,3437866695
122,5753791030
101,5371984021
7,274010452
157,8019217510
130,5927061748
89,4823028566


In [216]:

personid_test_df['PERSON_ID'] = personid_test_df.PERSON_ID.astype(str)

In [217]:
pred_prob_df.insert(0, 'PERSON_ID', personid_test_df, True)

In [218]:
type(personid_list_test)

pandas.core.series.Series

In [219]:
personid_list_test.dtype

dtype('int64')

In [220]:
pred_prob_df

,PERSON_ID,PERSON_ID,PERSON_ID,PERSON_ID,PERSON_ID,PERSON_ID,AB_AMF,AB_AOF,AB_CCARDS,AB_CWN,...,PROFFEE,PROFFEES,SAVING,SAVINGS,SCTB,SCTT,SDDB,ST,TFR,VAT
0,NaN,NaN,NaN,NaN,NaN,NaN,0.029727,0.054557,0.030540,0.021960,...,0.004080,0.002595,0.005581,0.003192,0.013725,0.044956,0.008615,0.002634,0.002802,0.002636
1,NaN,NaN,NaN,NaN,NaN,NaN,0.029727,0.054557,0.030540,0.021960,...,0.004080,0.002595,0.005581,0.003192,0.013725,0.044956,0.008615,0.002634,0.002802,0.002636
2,NaN,NaN,NaN,NaN,NaN,NaN,0.060543,0.075745,0.015274,0.019422,...,0.001532,0.001214,0.004005,0.002374,0.011499,0.022258,0.014069,0.001916,0.043033,0.001188
3,NaN,NaN,NaN,NaN,NaN,NaN,0.029727,0.054557,0.030540,0.021960,...,0.004080,0.002595,0.005581,0.003192,0.013725,0.044956,0.008615,0.002634,0.002802,0.002636
4,156312464,1.563125e+08,1.563125e+08,1.563125e+08,1.563125e+08,1.563125e+08,0.008986,0.044365,0.009365,0.019623,...,0.005525,0.001805,0.005056,0.002642,0.008259,0.008265,0.019283,0.008660,0.014475,0.001615
5,274010452,2.740105e+08,2.740105e+08,2.740105e+08,2.740105e+08,2.740105e+08,0.142558,0.193790,0.005284,0.005555,...,0.012495,0.004334,0.007647,0.006430,0.002930,0.003541,0.003641,0.010195,0.019062,0.003949
6,NaN,NaN,NaN,NaN,NaN,NaN,0.015961,0.046000,0.006743,0.004162,...,0.134548,0.018411,0.008839,0.006237,0.002986,0.005128,0.001817,0.035615,0.006655,0.017108
7,274010452,2.740105e+08,2.740105e+08,2.740105e+08,2.740105e+08,2.740105e+08,0.019297,0.016236,0.006904,0.008591,...,0.000419,0.001548,0.001921,0.004007,0.005602,0.008831,0.006946,0.003144,0.009593,0.001482
8,274010452,2.740105e+08,2.740105e+08,2.740105e+08,2.740105e+08,2.740105e+08,0.003003,0.004237,0.004414,0.003452,...,0.002397,0.003364,0.001295,0.002171,0.002998,0.005696,0.002036,0.002072,0.002028,0.002911
9,NaN,NaN,NaN,NaN,NaN,NaN,0.005578,0.009742,0.002984,0.001866,...,0.026208,0.018411,0.003843,0.009302,0.001429,0.002084,0.000900,0.015145,0.003066,0.064479


In [42]:

for i in pred:
    print(i)
for j in prob_pred:
    print("j value"+str(j))

ATM                           
ATM                           
ACHREV                        
ATM                           
AB_AOF                        
AB_AMF                        
PROFFEE                       
ACHPIF                        
COTOKSFA                      
ACCOPEN                       
CREDIT_CARDS                  
LTF                           
PROFFEE                       
DOM_MIN                       
ACCOPEN                       
INFUNDXFER                    
AB_AOF                        
COTOKSFA                      
ACCOPEN                       
ATM                           
ACHREV                        
ACHCR                         
ATM                           
ACHRET                        
ACCOPEN                       
COTOKSFA                      
COTOKSFA                      
ATM                           
COTOKSFA                      
AB_AOF                        
ACHPIF                        
ACCMAINT                      
ATM     

In [21]:
from sklearn.svm import SVC
svc =SVC(probability=False)
svc.fit(X_train,y_train)
pred=svc.predict(X_test)
pred
#pred=svc.predict_proba(X_test)
#print([i[1] for i in pred])

c:\users\reraveen\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array(['AB_AOF                        ', 'AB_AOF                        ',
       'AB_AOF                        ', 'AB_AOF                        ',
       'AB_AOF                        ', 'AB_AOF                        ',
       'AB_AOF                        ', 'ACHRET                        ',
       'ERIBDR                        ', 'INFUNDXFER                    ',
       'SAVINGS                       ', 'LMF                           ',
       'AB_AOF                        ', 'AB_AOF                        ',
       'INFUNDXFER                    ', 'INFUNDXFER                    ',
       'AB_AOF                        ', 'ERIBDR                        ',
       'INFUNDXFER                    ', 'AB_AOF                        ',
       'AB_AOF                        ', 'AB_AOF                        ',
       'AB_AOF                        ', 'AB_AOF                        ',
       'INFUNDXFER                    ', 'ERIBDR                        ',
       'ERIBDR           

In [22]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
                                precision    recall  f1-score   support

AB_AOF                               0.00      0.00      0.00       0.0
AB_CWN                               0.00      0.00      0.00       1.0
ACCOPEN                              0.00      0.00      0.00       1.0
ACHCR                                0.00      0.00      0.00       1.0
ACHRET                               0.00      0.00      0.00       0.0
ADMINFEE                             0.00      0.00      0.00       1.0
ATM                                  0.00      0.00      0.00       2.0
BUNDLE                               0.00      0.00      0.00       1.0
CDPTF                                0.00      0.00      0.00       2.0
CHEQUE_BOOK                          0.00      0.00      0.00       1.0
CHQBK                                0.00      0.00      0.00       2.0
CHQTFR         

c:\users\reraveen\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\reraveen\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

Accuracy of Logistic regression classifier on training set: 0.23
Accuracy of Logistic regression classifier on test set: 0.00


In [24]:
X_test.shape

(44, 46)

In [25]:
pred.shape

(44,)

In [26]:
y_test.shape

(44, 1)

In [27]:
y_train.shape

(131, 1)

In [45]:
type(y_train)

pandas.core.frame.DataFrame

In [46]:
y_train.columns.tolist()

['PRICEITEM_CD']

In [49]:
y_train.PRICEITEM_CD.unique()

array(['ACHREV                        ', 'ACHPIF                        ',
       'SCTB                          ', 'AB_CWN                        ',
       'LTF                           ', 'DRELECTR                      ',
       'CDMF                          ', 'ACCMAINT                      ',
       'DEBIT_CARDS                   ', 'ACHRET                        ',
       'CPYSTMT                       ', 'LMF                           ',
       'OBIF                          ', 'COTOKSFA                      ',
       'AB_AMF                        ', 'GL_STD                        ',
       'DEDEBIT                       ', 'ONDWT                         ',
       'ERIBDR                        ', 'MONSERV                       ',
       'PP_REG_MX_100                 ', 'DOMGROSS                      ',
       'ATM                           ', 'DISCT_KS                      ',
       'PROFFEES                      ', 'DOMNET                        ',
       'INFUNDXFER       

In [66]:
X_train.shape

(131, 46)

In [ ]:
y_test

In [ ]:
pd.DataFrame(pred)